## If you want to download datasets and trained models for convenience, go to this [link](https://drive.google.com/drive/folders/1Ltqj8wiuXxrl1p_KzrzfaJkn2EDfI-M4?usp=sharing). Download file named lda topic modelling.zip and unzip it

In [2]:
%config IPCompleter.use_jedi = False

In [3]:
import pandas as pd
import gensim
import spacy
import nltk
from bs4 import BeautifulSoup
import unidecode
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import word2vec
import numpy as np
from nltk.corpus import stopwords

import gensim.corpora as corpora
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
%run contractions.ipynb

In [3]:
# df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df = pd.read_json("D:/pytorch dataset and model/newsgroups.json")

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
content = df.content.values

# PREPROCESSING

## removing html tags and new line character

In [6]:
raw_html = content[0]
re.sub('\s+', ' ', raw_html)

"From: lerxst@wam.umd.edu (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15 I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL ---- brought to you by your neighborhood Lerxst ---- "

In [7]:
def removing_new_line(text):
    text = re.sub('\s+', ' ', text)
    return text
content = list(map(removing_new_line, content))

In [8]:
def remove_accent_char(text):
    res = unidecode.unidecode(text)
    return res
content = list(map(remove_accent_char, content))

In [9]:
c_re = re.compile('(%s)' % '|'.join(CONTRACTION_MAP.keys()))
def expand_contractions(s, contractions_dict=CONTRACTION_MAP):
    def replace(match):
        return contractions_dict[match.group(0)]
    return c_re.sub(replace, s)
content = list(map(expand_contractions, content))

In [10]:
def remove_email(text):
    text = re.sub('\S+@\S+', ' ', text)
    return text
content = list(map(remove_email, content))

In [11]:
content = list(map(lambda text: text.lower(), content))

In [12]:
def remove_posting_host(text):
    text = re.sub(r'\bnntp\S+\s\S+', ' ', text)
    return text
content = list(map(remove_posting_host, content))

In [13]:
df.content = content

In [15]:
def remove_subject(text):
    text = re.sub(r'\Afrom.+\bre:', ' ', text)
    text = re.sub(r'\Afrom.+\bsubject:', ' ', text)
    return text
content = list(map(remove_subject, content))

## remove sentences that its length is so much long (>5000)

In [16]:
remove_arr = []
for ind, cont in enumerate(df.target):
    if cont == 2:
        if len(content[ind]) > 5000:
            remove_arr.append(ind)


In [17]:
for ind, cont in enumerate(content):
    if ind in remove_arr:
        content.remove(cont)
len(content)

11289

In [18]:
def remove_special_characters(text):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, ' ', text)
    return text
content = list(map(remove_special_characters, content))

In [19]:
def remove_spacing(text):
    res = " ".join(text.split())
    return res
content = list(map(remove_spacing, content))

In [5]:
def tokenizer(text):
    text = nltk.word_tokenize(text)
    return text
token_sent = list(map(tokenizer, content))

In [ ]:
i =0
tok_rm_sent = []
for i in range(len(token_sent)):
# def remove_stopwords(token_words):
    tok_rm_sent.append([word for word in token_sent[i] if word not in set(stopwords.words("english"))])
    print(i)
    i = i + 1
# token_sent = list(map(remove_stopwords, token_sent))



## Save dataset after preprocessing

In [82]:
with open('my_file.txt', 'w') as filehandle:
    for listitem in token_sent:
        filehandle.write('%s\n' % listitem)

In [88]:
with open("my_file.txt" , 'r') as f:
    token_sent = f.readlines()

## Tokenize sentences

In [6]:
def removing_new_line(text):
    text = re.sub('\s+', ' ', text)
    return text
token_sent = list(map(removing_new_line, token_sent))
def remove_spacing(text):
    res = " ".join(text.split())
    return res
token_sent = list(map(remove_spacing, token_sent))
def tokenizer(text):
    text = nltk.word_tokenize(text)
    return text
token_sent = list(map(tokenizer, token_sent))

## Make bigram and trigram model

In [108]:
bigram = gensim.models.Phrases(token_sent, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[token_sent], threshold=10)  

In [111]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [121]:
def make_bigram(text):
    return bigram_mod[text]

In [126]:
bigram_sent = list(map(make_bigram, token_sent))

In [131]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    text = [lemmatizer.lemmatize(word) for word in text]
    return text
bigram_sent = list(map(lemmatize_text, bigram_sent))

## Create id2Word Dictionary

In [135]:
id2Word = corpora.Dictionary(bigram_sent)
id2Word

In [144]:
corpus = [id2Word.doc2bow(text) for text in bigram_sent]

## Create lda model

In [145]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2Word,
                                           num_topics=20, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model

In [4]:
modelname = "lda_model"
# lda_model.save(modelname)

In [5]:
lda_model = models.LdaModel.load(modelname)
lda_model

In [14]:
lda_model.print_topics(3)

[(8,
  '0.047*"cheap" + 0.034*"brown" + 0.025*"specification" + 0.016*"worried" + 0.014*"catalog" + 0.014*"import" + 0.012*"waste" + 0.011*"ab" + 0.009*"tech_support" + 0.009*"orange"'),
 (11,
  '0.024*"people" + 0.015*"one" + 0.015*"god" + 0.012*"say" + 0.012*"would" + 0.010*"think" + 0.010*"mean" + 0.009*"believe" + 0.009*"fact" + 0.008*"may"'),
 (5,
  '0.023*"would" + 0.020*"one" + 0.018*"writes" + 0.015*"get" + 0.014*"organization" + 0.012*"like" + 0.012*"line" + 0.011*"know" + 0.010*"time" + 0.010*"could"')]

## Perplexity: a measure of how good the model is. lower the better.

In [156]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -17.033826500085


In [159]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=bigram_sent, dictionary=id2Word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48978657899913614
